# 视觉提示训练


## 数据集准备

## visual prompt 训练
训练 yolo标注格式的目标检测数据集

```
# For visual prompt, because only SAVPE is trained, we can adopt the detection pipeline with less training time

# First, obtain the detection model
python tools/convert_segm2det.py
# Then, train the SAVPE module
python train_vp.py
# After training, please use tools/get_vp_segm.py to add the segmentation head
# python tools/get_vp_segm.py
```

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 

# 设置 HTTP 和 HTTPS 代理
os.environ["http_proxy"] = "***"
os.environ["https_proxy"] = "***"

## convert segm2det

In [4]:
# from ultralytics import YOLOE
# import torch

# det_model = YOLOE("yoloe-v8l.yaml")

# state = torch.load("./pretrain/yoloe-v8l-seg.pt")

# det_model.load(state["model"])
# det_model.save("./pretrain/yoloe-v8l-seg-det.pt")

# model = YOLOE("./pretrain/yoloe-v8l-seg-det.pt")
# print(model.args)


## train

In [5]:
from ultralytics import YOLOE
from ultralytics.models.yolo.yoloe.train_seg_vp import YOLOESegVPTrainer
from ultralytics.models.yolo.yoloe.train_vp import YOLOEVPTrainer
import os
from ultralytics.nn.tasks import guess_model_scale
from ultralytics.utils import yaml_load, LOGGER

os.environ["PYTHONHASHSEED"] = "0"
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [6]:

# yolo格式的目标检测数据集
data = dict(
    train=dict(
        yolo_data=["/root/project/research/Yolo/yoloe/ultralytics/cfg/datasets/glass.yaml"],
    ),
    val=dict(yolo_data=["/root/project/research/Yolo/yoloe/ultralytics/cfg/datasets/glass.yaml"]),
)

# yolo格式的实例分割数据集
"""
data = dict(
    train=dict(
        yolo_data=["/root/project/research/Yolo/yoloe/ultralytics/cfg/datasets/custom.yaml"],
    ),
    val=dict(yolo_data=["/root/project/research/Yolo/yoloe/ultralytics/cfg/datasets/custom.yaml"]),
)
"""

model_path = "yoloe-v8l.yaml"

scale = guess_model_scale(model_path)
cfg_dir = "ultralytics/cfg"
default_cfg_path = f"{cfg_dir}/default.yaml"
extend_cfg_path = f"{cfg_dir}/{scale}_train.yaml"
defaults = yaml_load(default_cfg_path)
extends = yaml_load(extend_cfg_path)
assert(all(k in defaults for k in extends))
LOGGER.info(f"Extends: {extends}")

Extends: {}


In [7]:
model = YOLOE("./pretrain/yoloe-v8l-seg-det.pt")
# YOLOESegVPTrainer => YOLOEVPTrainer

head_index = len(model.model.model) - 1
freeze = list(range(0, head_index))
for name, child in model.model.model[-1].named_children():
    if 'savpe' not in name:
        freeze.append(f"{head_index}.{name}")

In [9]:
# For s/m, please set lr0=8e-3
model.train(data=data, batch=32, epochs=2, **extends, close_mosaic=2, \
    optimizer='AdamW', lr0=16e-3, warmup_bias_lr=0.0, \
        weight_decay=0.025, momentum=0.9, workers=4, \
        trainer=YOLOEVPTrainer, device='0', freeze=freeze, load_vp=True)